In [8]:
def F(x: int, y: int, z: int) -> int:
    return ((x & ~y) ^ (y & ~z) ^ (z & ~x)) ^ (
        ((x + y) * (y + z) + (x + z)) & 0xFFFF_FFFF_FFFF_FFFF_FFFF_FFFF_FFFF_FFFF
    )


def _block_hash(a: int, b: int, c: int, d: int) -> int:
    x, y, z, w = F(a, b, c), F(b, c, d), F(c, d, a), F(d, a, b)
    return (a ^ b ^ c ^ d ^ x ^ y ^ z ^ w) ^ 0xFFFF_FFFF_FFFF_FFFF_FFFF_FFFF_FFFF_FFFF


def _hash(blocks: list[int]) -> int:
    length = len(blocks)
    if length % 4 != 0:
        padding = 4 - length % 4
        blocks += [0] * padding
        length += padding
    if length == 4:
        return _block_hash(*blocks)
    else:
        block_size = length // 4
        h1 = _hash(blocks[:block_size])
        h2 = _hash(blocks[block_size : block_size * 2])
        h3 = _hash(blocks[block_size * 2 : block_size * 3])
        h4 = _hash(blocks[block_size * 3 :])
        return _block_hash(h1, h2, h3, h4)


def bytes2blocks(data: bytes, block_size=16) -> list[int]:
    if len(data) % block_size != 0:
        data += b"\x00" * (block_size - len(data) % block_size)
    return [
        int.from_bytes(data[i : i + block_size], "little")
        for i in range(0, len(data), block_size)
    ]


def hash(*data: list[bytes]) -> int:
    return _hash(bytes2blocks(b"".join(data)))

In [18]:
from itertools import permutations
for i in permutations([b"1",b"2",b"3",b"4"]):
    print(b"".join(i),hash(*(x*16 for x in i)))

b'1234' 340261516600152070536831223370151361527
b'1243' 233238878006135035263925232372580928759
b'1324' 329646269423307513812120041455026698231
b'1342' 233238878006135035263925232372580928759
b'1423' 329646269423307513812120041455026698231
b'1432' 340261516600152070536831223370151361527
b'2134' 233238878006135035263925232372580928759
b'2143' 340261516600152070536831223370151361527
b'2314' 329646269423307513812120041455026698231
b'2341' 340261516600152070536831223370151361527
b'2413' 329646269423307513812120041455026698231
b'2431' 233238878006135035263925232372580928759
b'3124' 233238878006135035263925232372580928759
b'3142' 329646269423307513812120041455026698231
b'3214' 340261516600152070536831223370151361527
b'3241' 329646269423307513812120041455026698231
b'3412' 340261516600152070536831223370151361527
b'3421' 233238878006135035263925232372580928759
b'4123' 340261516600152070536831223370151361527
b'4132' 329646269423307513812120041455026698231
b'4213' 23323887800613503526392523237258

观察易知，存在部分交换，可以使hash不变

In [ ]:
from string import ascii_letters
from hashlib import md5
from itertools import product
target = "435c285c011ac2c93a02cef74edb6424"
salt = "sHAv"
for x in product(ascii_letters,repeat=4):
    s = "".join(x)+salt
    h = md5(s.encode()).hexdigest()
    if h == target:
        print(s)
        break


In [ ]:
from base64 import b64decode,b64encode
data = b64decode("RoZqav2gsy4OrPb9w0dd80XAsmQrbq0UGSvg2H9TYeG9PoepDdhJSCDCXdCc5wotHi6Ua3dN/4rQQ6L6RZpVBDKimKEMnccmz2R6h/6YYjGb7yURhO2bvGr9+JeUjZ/x/9o3IC+gvlJ4TnOGXPi6MKdhr4yZ7OC8UT6Ht9IKZhEzOmw6u9FXr95pnINhdVaqYspJzI21NM7JctkOWrdflIqJG1HC3qxpQ9Xnqn3nfiQmKx4AKrF10F5VhMDPy/ClubMSZVO2ylQ8w+pX2KN0ze2rOrlo3Ogt9+DgAxJokLaSSkwfjylRzVvjSN/4Tvj2GFCkPpFb33QJhF42OrnUpw==")
payload = b""
for x in range(4):
    a = data[64*x:64*x+64]
    for y in range(3,-1,-1):
        b = a[16*y:16*y+16]
        payload += b
b64encode(payload)